<a href="https://colab.research.google.com/github/gevenbly/TensorAlgs/blob/main/mod_binary_MERA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def expand_dims(tensor, new_dims):
  """ 
  Expand the dims of a tensor by padding with zeros.
  """
  old_dims = tensor.shape
  dim_expand = [(old_dims[k], new_dims[k]) for k in range(tensor.ndim)]

  return np.pad(tensor, dim_expand)

In [ ]:
def tprod(*tensor_list, do_matricize=True):
  """ 
  Tensor product for operators. Expands the functionality of `kron` to accept 
  tensors rather than just matrices, and to accept and arbitrary number of 
  inputs. The index ordering is defined such that if the inputs are Hermtian 
  matrices the the output tensor can be reshaped into a Hermitian matrix.
  """
  
  # take kron of each input sequentially
  shapes_L = []
  shapes_R = []
  final_tensor = np.array(1.0, dtype=float)
  for tensor in tensor_list:
    shapes_L = shapes_L + list(tensor.shape[:(tensor.ndim//2)])
    shapes_R = shapes_R + list(tensor.shape[(tensor.ndim//2):])
    final_tensor = np.kron(final_tensor, matricize(tensor))

  if do_matricize:
    return final_tensor
  else:
    return final_tensor.reshape(shapes_L + shapes_R)

In [ ]:
def matricize(tensor, partition=None):
  """ Matricize an input tensor across some left/right partition. """
  
  if partition is None:
    partition = tensor.ndim // 2

  size_L = np.prod(tensor.shape[:partition])
  size_R = np.prod(tensor.shape[partition:])
 
  return tensor.reshape(size_L, size_R)

In [ ]:
def orthogonalize(tensor, partition=None):
  """ Orthogonalize an input tensor across some left/right partition. """

  tshape = tensor.shape
  ut, st, vt = LA.svd(matricize(tensor, partition=partition), 
                      full_matrices=False)

  return (ut @ vt).reshape(tshape)


In [2]:
def define_ham(blocksize):
  """ Define Hamiltonian (quantum critical Ising) """

  # define Pauli matrices
  sX = np.array([[0, 1], [1, 0]], dtype=float)
  sZ = np.array([[1, 0], [0, -1]], dtype=float)

  # define Ising local Hamiltonian
  ham_orig = (tprod(sX, sX) - 0.5*tprod(sZ, np.eye(2)) - 
              0.5*tprod(np.eye(2), sZ))

  # shift Hamiltonian to ensure negative defined
  en_shift = max(LA.eigh(ham_orig)[0])
  ham_loc = ham_orig - en_shift*np.eye(4)

  # define block Hamiltonians 
  d0 = 2 # initial local dim
  d1 = d0**blocksize # local dim after blocking

  if blocksize==3:
    ham_block = (1.0*tprod(np.eye(d0**1), ham_loc, np.eye(d0**3)) + 
                 1.0*tprod(np.eye(d0**2), ham_loc, np.eye(d0**2)) +
                 1.0*tprod(np.eye(d0**3), ham_loc, np.eye(d0**1))
                 ).reshape(d0*np.ones(12, dtype=int))
    hamAB_init = ham_block.transpose(0,1,2,5,4,3,6,7,8,11,10,9
                                    ).reshape(d1, d1, d1, d1)
    hamBA_init = ham_block.transpose(2,1,0,3,4,5,8,7,6,9,10,11
                                    ).reshape(d1, d1, d1, d1)
  elif blocksize==4:
    ham_block = (0.5*tprod(np.eye(d0**1), ham_loc, np.eye(d0**5)) + 
                1.0*tprod(np.eye(d0**2), ham_loc, np.eye(d0**4)) + 
                1.0*tprod(np.eye(d0**3), ham_loc, np.eye(d0**3)) +
                1.0*tprod(np.eye(d0**4), ham_loc, np.eye(d0**2)) +
                0.5*tprod(np.eye(d0**5), ham_loc, np.eye(d0**1))
                ).reshape(d0*np.ones(16, dtype=int))
    hamAB_init = ham_block.transpose(0,1,2,3,7,6,5,4,8,9,10,11,15,14,13,12
                                    ).reshape(d1, d1, d1, d1)
    hamBA_init = ham_block.transpose(3,2,1,0,4,5,6,7,11,10,9,8,12,13,14,15
                                    ).reshape(d1, d1, d1, d1)
  
  return hamAB_init, hamBA_init, en_shift

In [ ]:
def initialize(chi, chimid, hamAB_init, hamBA_init):
  """ Initialize the MERA tensors """

  # Initialize the MERA tensors
  d1 = hamAB_init.shape[0]
  iso_temp = orthogonalize(np.random.rand(d1, min(chimid, d1)))
  uC = [tprod(iso_temp, iso_temp, do_matricize=False)]
  wC = [orthogonalize(np.random.rand(d1, uC[0].shape[2], chi), partition=2)]
  vC = [orthogonalize(np.random.rand(d1, uC[0].shape[2], chi), partition=2)]
  for k in range(layers-1):
    iso_temp = orthogonalize(np.random.rand(chi, chimid))
    uC.append(tprod(iso_temp, iso_temp, do_matricize=False))
    wC.append(orthogonalize(np.random.rand(chi, chimid, chi), partition=2))
    vC.append(orthogonalize(np.random.rand(chi, chimid, chi), partition=2))
  
  # initialize density matrices and effective Hamiltonians
  rhoAB = [0]
  rhoBA = [0]
  hamAB = [hamAB_init]
  hamBA = [hamBA_init]
  for k in range(layers):
    rhoAB.append(np.eye((chi**2, chi**2)).reshape(chi, chi, chi, chi))
    rhoBA.append(np.eye((chi**2, chi**2)).reshape(chi, chi, chi, chi))
    hamAB.append(np.zeros((chi, chi, chi, chi)))
    hamBA.append(np.zeros((chi, chi, chi, chi)))
  
  return hamAB, hamBA, wC, vC, uC, rhoAB, rhoBA

In [ ]:
def define_networks(hamAB, hamBA, wC, vC, uC, rhoAB, rhoBA):
  """ Define and plot all principle networks """

  # Define the `M` principle network
  connects_M = [[3,5,9], [1,5,7], [1,2,3,4], [4,6,10], [2,6,8], [7,8,9,10]]
  tensors_M = [vC[1], vC[1], hamBA[1], wC[1], wC[1], rhoAB[2]]
  order_M = ncon_solver(tensors_M, connects_M)[0]
  dims_M = [tensor.shape for tensor in tensors_M]
  names_M = ['v', 'v', 'hBA', 'w', 'w', 'rhoAB']
  coords_M = [(-0.5,1),(-0.5,-1), (-0.3,-0.2,0.3,0.2),(0.5,1),(0.5,-1),(0.2)]
  colors_M = [0,0,1,2,2,3]

  # Define the `L` principle network
  connects_L = [[3,6,13], [1,8,11], [4,5,6,7], [2,5,8,9], [1,2,3,4], 
                [10,7,14], [10,9,12], [11,12,13,14]]
  tensors_L = [wC[1], wC[1], uC[1], uC[1], hamAB[1], vC[1], vC[1], rhoBA[2]]
  order_L = ncon_solver(tensors_L, connects_L)[0]
  dims_L = [tensor.shape for tensor in tensors_L]
  names_L = ['w', 'w', 'u', 'u', 'hAB', 'v', 'v', 'rhoBA']
  coords_L = [(-0.5, 1.5), (-0.5, -1.5), (-0.3,0.5,0.3,0.9), (-0.3,-0.5,0.3,-0.9), 
              (-0.6,-0.2,-0.1,0.2), (0.5, 1.5), (0.5, -1.5), (0.2)]
  colors_L = [2,2,4,4,1,0,0,3]

  # Define the `C` principle network
  connects_C = [[5,6,13], [5,9,11], [3,4,6,8], [1,2,9,10], [1,2,3,4], [7,8,14],
                [7,10,12], [11,12,13,14]]
  tensors_C = [wC[1], wC[1], uC[1], uC[1], hamBA[1], vC[1], vC[1], rhoBA[2]]
  order_C = ncon_solver(tensors_C, connects_C)[0]
  dims_C = [tensor.shape for tensor in tensors_C]
  names_C = ['w', 'w', 'u', 'u', 'hBA', 'v', 'v', 'rhoBA']
  coords_C = [(-0.5, 1.5), (-0.5, -1.5), (-0.3,0.5,0.3,0.9), (-0.3,-0.5,0.3,-0.9), 
              (-0.3,-0.2,0.3,0.2), (0.5, 1.5), (0.5, -1.5), (0.2)]
  colors_C = [2,2,4,4,1,0,0,3]

  # Define the `R` principle network
  connects_R = [[10,6,13], [10,8,11], [5,3,6,7], [5,1,8,9], [2,1,4,3], [4,7,14],
                [2,9,12], [11,12,13,14]]
  tensors_R = [wC[1], wC[1], uC[1], uC[1], hamAB[1], vC[1], vC[1], rhoBA[2]]
  order_R = ncon_solver(tensors_R, connects_R)[0]
  dims_R = [tensor.shape for tensor in tensors_R]
  names_R = ['w', 'w', 'u', 'u', 'hAB', 'v', 'v', 'rhoBA']
  coords_R = [(-0.5, 1.5), (-0.5, -1.5), (-0.3,0.5,0.3,0.9), (-0.3,-0.5,0.3,-0.9), 
              (0.6,-0.2,0.1,0.2), (0.5, 1.5), (0.5, -1.5), (0.2)]
  colors_R = [2,2,4,4,1,0,0,3]

  # Plot all principle networks
  fig = plt.figure(figsize=(24,24))
  figM = draw_network(connects_M, order=order_M, dims=dims_M, coords=coords_M, 
                      names=names_M, colors=colors_M, title='M-diagrams', 
                      draw_labels=False, show_costs=True, legend_extend=2.5, 
                      fig=fig, subplot=141, env_pad=(-0.4,-0.4))
  figL = draw_network(connects_L, order=order_L, dims=dims_L, coords=coords_L, 
                      names=names_L, colors=colors_L, title='L-diagrams', 
                      draw_labels=False, show_costs=True, legend_extend=2.5, 
                      fig=fig, subplot=142, env_pad=(-0.4,-0.4))
  figC = draw_network(connects_C, order=order_C, dims=dims_C, coords=coords_C, 
                      names=names_C, colors=colors_C, title='C-diagrams', 
                      draw_labels=False, show_costs=True, legend_extend=2.5, 
                      fig=fig, subplot=143, env_pad=(-0.4,-0.4))
  figR = draw_network(connects_R, order=order_R, dims=dims_R, coords=coords_R, 
                      names=names_R, colors=colors_R, title='R-diagrams', 
                      draw_labels=False, show_costs=True, legend_extend=2.5, 
                      fig=fig, subplot=144, env_pad=(-0.4,-0.4))

  # Store `connects` and `order` in a dict for later use
  network_dict = {'connects_M': connects_M, 'order_M': order_M,
                  'connects_L': connects_L, 'order_L': order_L,
                  'connects_C': connects_C, 'order_C': order_C,
                  'connects_R': connects_R, 'order_R': order_R,}

  return network_dict

In [ ]:
def lift_hamiltonian(hamAB, hamBA, w, v, u, rhoAB, rhoBA, network_dict):
  """ Lift the Hamiltonian through one MERA layer """

  hamAB_lift = xcon([v, v, hamBA, w, w, rhoAB], 
                    network_dict['connects_M'], 
                    order=network_dict['order_M'], which_envs=5)

  hamBA_temp0 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                     network_dict['connects_L'], 
                     order=network_dict['order_L'], which_envs=7)

  hamBA_temp1 = xcon([w, w, u, u, hamBA, v, v, rhoBA], 
                     network_dict['connects_C'], 
                     order=network_dict['order_C'], which_envs=7)

  hamBA_temp2 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                     network_dict['connects_R'], 
                     order=network_dict['order_R'], which_envs=7)
  
  hamBA_lift = hamBA_temp0 + hamBA_temp1 + hamBA_temp2

  return hamAB_lift, hamBA_lift

In [ ]:
def lower_density(hamAB, hamBA, w, v, u, rhoAB, rhoBA, network_dict):
  """ Lower the density matrix through one MERA layer """

  rhoBA_temp0 = xcon([v, v, hamBA, w, w, rhoAB], 
                     network_dict['connects_M'], 
                     order=network_dict['order_M'], which_envs=2)

  rhoAB_temp0 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                     network_dict['connects_L'], 
                     order=network_dict['order_L'], which_envs=4)

  rhoBA_temp1 = xcon([w, w, u, u, hamBA, v, v, rhoBA], 
                     network_dict['connects_C'], 
                     order=network_dict['order_C'], which_envs=4)
  
  rhoAB_temp1 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                     network_dict['connects_R'], 
                     order=network_dict['order_R'], which_envs=4)
  
  rhoAB_lower = 0.5*(rhoAB_temp0 + rhoAB_temp1)
  rhoBA_lower = 0.5*(rhoBA_temp0 + rhoBA_temp1)

  return rhoAB_lower, rhoBA_lower

In [ ]:
def optimize_w(hamAB, hamBA, w, v, u, rhoAB, rhoBA, network_dict):
  """ Optimise the `w` isometry """

  w_env0 = xcon([v, v, hamBA, w, w, rhoAB], network_dict['connects_M'], 
                order=network_dict['order_M'], which_envs=3)
  
  w_env1 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_L'], order=network_dict['order_L'], 
                which_envs=0)
  
  w_env2 = xcon([w, w, u, u, hamBA, v, v, rhoBA], 
                network_dict['connects_C'], order=network_dict['order_C'], 
                which_envs=0)
  
  w_env3 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_R'], order=network_dict['order_R'], 
                which_envs=0)
  
  w_out = orthogonalize(w_env0 + w_env1 + w_env2 + w_env3, partition=2)

  return w_out

In [ ]:
def optimize_v(hamAB, hamBA, w, v, u, rhoAB, rhoBA, network_dict):
  """ Optimise the `v` isometry """

  v_env0 = xcon([v, v, hamBA, w, w, rhoAB], network_dict['connects_M'], 
                order=network_dict['order_M'], which_envs=0)
  
  v_env1 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_L'], order=network_dict['order_L'], 
                which_envs=5)
  
  v_env2 = xcon([w, w, u, u, hamBA, v, v, rhoBA], 
                network_dict['connects_C'], order=network_dict['order_C'], 
                which_envs=5)
  
  v_env3 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_R'], order=network_dict['order_R'], 
                which_envs=5)
  
  v_out = orthogonalize(v_env0 + v_env1 + v_env2 + v_env3, partition=2)

  return v_out

In [ ]:
def optimize_u(hamAB, hamBA, w, v, u, rhoAB, rhoBA, network_dict):
  """ Optimise the `w` isometry """
  
  u_env0 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_L'], order=network_dict['order_L'], 
                which_envs=2)
  
  u_env1 = xcon([w, w, u, u, hamBA, v, v, rhoBA], 
                network_dict['connects_C'], order=network_dict['order_C'], 
                which_envs=2)
  
  u_env2 = xcon([w, w, u, u, hamAB, v, v, rhoBA], 
                network_dict['connects_R'], order=network_dict['order_R'], 
                which_envs=2)
  
  u_out = orthogonalize(u_env0 + u_env1 + u_env2, partition=2)

  return u_out